In [ ]:
from google.colab import drive
drive.mount('/gdrive')
!rm -rf /content/gdrive
!ln -s "/gdrive/My Drive" /content/gdrive
%cd /content/gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive


In [ ]:
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM
import pandas as pd
import numpy as np
import re
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split

In [ ]:
formspring=pd.read_csv("/content/gdrive/formspring_data_1_###_200805_labeling.csv", encoding='latin1')

In [ ]:
formspring.head(10)

,sentence,sentiment
0,Q: what&#039;s your favorite song? :D<br>A: I ...,non-bully
1,Q: <3<br>A: </3 ? haha jk! <33,non-bully
2,Q: &quot;hey angel you duh sexy&quot;<br>A: R...,non-bully
3,Q: (:<br>A: ;(,non-bully
4,Q: ******************MEOWWW*******************...,non-bully
5,Q: any makeup tips? i suck at doing my makeup ...,non-bully
6,Q: Apriiiiiiiiiiiill!!! I miss uuuu! It&#039;s...,non-bully
7,Q: Are you a morning or night person?<br>A: Ni...,non-bully
8,Q: are you a trusting person?<br>A: alreadi an...,non-bully
9,Q: are you a trusting person?<br>A: Yes veryy ...,non-bully


In [ ]:
data = formspring[['sentence','sentiment']]

In [ ]:
import string
data['sentence'] = data['sentence'].apply(lambda x: x.replace('Q:',' ').replace('A:',' ').replace('<br>',' '))
data['sentence'] = data['sentence'].apply(lambda x: x.lower())
data['sentence'] = data['sentence'].apply((lambda x: re.sub(r"(https?|http)://[-a-zA-Z0-9+&@#/%?=~_|!:,.;]*[-a-zA-Z0-9+&@#/%=~_|]", "", x)))
data['sentence'] = data['sentence'].apply((lambda x: re.sub(r"&#039;", "\'", x)))
data['sentence'] = data['sentence'].apply((lambda x: re.sub(r"&quot;", "", x)))
data['sentence'] = data['sentence'].apply((lambda x: re.sub(r"&amp;", "", x)))
print(data[ data['sentiment'] == 'non-bully'].size)
print(data[ data['sentiment'] == 'bully'].size)

23994
1552


In [ ]:
data.head(10)

,sentence,sentiment
0,what's your favorite song? :d i like too m...,non-bully
1,<3 </3 ? haha jk! <33,non-bully
2,hey angel you duh sexy really?!?! thanks?...,non-bully
3,(: ;(,non-bully
4,******************meowww********************...,non-bully
5,any makeup tips? i suck at doing my makeup l...,non-bully
6,apriiiiiiiiiiiill!!! i miss uuuu! it's emma ...,non-bully
7,are you a morning or night person? night 4...,non-bully
8,are you a trusting person? alreadi answrd,non-bully
9,are you a trusting person? yes veryy trust...,non-bully


In [ ]:
tokenizer = Tokenizer(split=' ')
tokenizer.fit_on_texts(data['sentence'].values)
X = tokenizer.texts_to_sequences(data['sentence'].values)
X = pad_sequences(X, maxlen=62)

In [ ]:
Y = pd.get_dummies(data['sentiment']).values

In [ ]:
print(Y[0])

[0 1]


In [ ]:
y = []
for i in range(12773):
  y.append(Y[i][1])

Y = np.asarray(y)

In [ ]:
print(Y[0])

1


In [ ]:
print(X.shape)

(12773, 62)


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)

In [ ]:
imdb=pd.read_csv("/content/gdrive/IMDB Dataset.csv")

In [ ]:
imdb.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [ ]:
data_imdb = imdb[['review','sentiment']]

In [ ]:
import string
data_imdb['review'] = data_imdb['review'].apply(lambda x: x.lower())
data_imdb['review'] = data_imdb['review'].apply(lambda x: "".join(l for l in x if l not in string.punctuation))

In [ ]:
X_imdb = tokenizer.texts_to_sequences(data_imdb['review'].values)
X_imdb = pad_sequences(X_imdb, maxlen=62)

In [ ]:
Y_imdb = pd.get_dummies(data_imdb['sentiment']).values

In [ ]:
print(Y_imdb[0])

[0 1]


In [ ]:
y_imdb = []

for i in range(50000):
  y_imdb.append(Y_imdb[i][1])

Y_imdb = np.asarray(y_imdb)

In [ ]:
print(Y_imdb[0])

1


In [ ]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open('/content/gdrive/glove.twitter.27B.50d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

In [ ]:
vocab_size = len(tokenizer.word_index) + 1

embedding_matrix = zeros((vocab_size, 50))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [ ]:
from keras.layers import Bidirectional, Flatten, Reshape, Lambda
from keras.layers.convolutional import Conv2D, MaxPooling2D 
from sklearn import metrics

embed_dim = 50
lstm_out = 50

model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, embed_dim, weights=[embedding_matrix], input_length = X.shape[1]))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(lstm_out, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 62, 50)            936200    
_________________________________________________________________
dropout (Dropout)            (None, 62, 50)            0         
_________________________________________________________________
bidirectional (Bidirectional (None, 62, 100)           40400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 62, 100)           0         
_________________________________________________________________
flatten (Flatten)            (None, 6200)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 6201      
Total params: 982,801
Trainable params: 982,801
Non-trainable params: 0
__________________________________________________

In [ ]:
model.fit(X_imdb, Y_imdb, epochs=10, batch_size=128)

Epoch 1/10
391/391 [==============================] - 29s 22ms/step - loss: 0.5912 - accuracy: 0.6682
Epoch 2/10
391/391 [==============================] - 8s 21ms/step - loss: 0.4125 - accuracy: 0.8090
Epoch 3/10
391/391 [==============================] - 8s 21ms/step - loss: 0.3709 - accuracy: 0.8323
Epoch 4/10
391/391 [==============================] - 8s 21ms/step - loss: 0.3327 - accuracy: 0.8530
Epoch 5/10
391/391 [==============================] - 8s 21ms/step - loss: 0.3146 - accuracy: 0.8621
Epoch 6/10
391/391 [==============================] - 8s 21ms/step - loss: 0.2959 - accuracy: 0.8735
Epoch 7/10
391/391 [==============================] - 8s 21ms/step - loss: 0.2757 - accuracy: 0.8818
Epoch 8/10
391/391 [==============================] - 8s 21ms/step - loss: 0.2635 - accuracy: 0.8882
Epoch 9/10
391/391 [==============================] - 8s 21ms/step - loss: 0.2492 - accuracy: 0.8946
Epoch 10/10
391/391 [==============================] - 8s 21ms/step - loss: 0.2363 - accur

In [ ]:
model.fit(X_train, Y_train, epochs=50, batch_size=128)

Epoch 1/50
80/80 [==============================] - 2s 22ms/step - loss: 0.2393 - accuracy: 0.9303
Epoch 2/50
80/80 [==============================] - 2s 21ms/step - loss: 0.1644 - accuracy: 0.9472
Epoch 3/50
80/80 [==============================] - 2s 21ms/step - loss: 0.1328 - accuracy: 0.9550
Epoch 4/50
80/80 [==============================] - 2s 21ms/step - loss: 0.1114 - accuracy: 0.9614
Epoch 5/50
80/80 [==============================] - 2s 21ms/step - loss: 0.0954 - accuracy: 0.9659
Epoch 6/50
80/80 [==============================] - 2s 20ms/step - loss: 0.0836 - accuracy: 0.9708
Epoch 7/50
80/80 [==============================] - 2s 20ms/step - loss: 0.0693 - accuracy: 0.9755
Epoch 8/50
80/80 [==============================] - 2s 21ms/step - loss: 0.0582 - accuracy: 0.9793
Epoch 9/50
80/80 [==============================] - 2s 20ms/step - loss: 0.0471 - accuracy: 0.9833
Epoch 10/50
80/80 [==============================] - 2s 21ms/step - loss: 0.0401 - accuracy: 0.9858
Epoch 11/

In [ ]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, Y_test)[1]))

80/80 [==============================] - 2s 5ms/step - loss: 0.3986 - accuracy: 0.9515

 테스트 정확도: 0.9515


In [ ]:
predict_test = (model.predict(X_test) >= 0.5) 

In [ ]:
from sklearn import metrics

print('Accuracy:\n',
      metrics.accuracy_score(Y_test, predict_test))

Accuracy:
 0.95146771037182


In [ ]:
print('\nPrecision:\n',
      metrics.precision_score(Y_test, predict_test, average=None))
print('\nRecall:\n',
      metrics.recall_score(Y_test, predict_test, average=None))
print('\nF-measure:\n',
      metrics.f1_score(Y_test, predict_test, average=None))


Precision:
 [0.57364341 0.97155812]

Recall:
 [0.51748252 0.97719735]

F-measure:
 [0.54411765 0.97436957]
